In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import socket

width, height = 1280, 720

cap = cv2.VideoCapture(0)
cap.set(3, width)
cap.set(4, height)

detector = HandDetector(detectionCon = 0.8)

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
serverAddressPort = ("127.0.0.1", 5052)

cx, cy, cz = 720, 360, 300
rx, ry, rz = 0, 0, 0
W, H, L = 5, 5, 5

newx, newy, newz = cx, cy, cz
newrx, newry, newrz = rx, ry, rz
newW, newH, newL = W, H, L

delx, dely, delz = 0, 0, 0
delrx, delry, delrz = 0, 0, 0
scale = 0

startPos1 = None
startPos2 = None
startPos3 = None
startDistance = None

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType = False)
    
    flag = 0
    if len(hands) == 1:
        # 移动
        if detector.fingersUp(hands[0]) == [1, 1, 1, 0, 0]:
            dis, _, _ = detector.findDistance(hands[0]['lmList'][8][0:2], hands[0]['lmList'][12][0:2], img)
            if dis < 50:
                flag = 1
                if startPos1 is None:
                    startPos1 = hands[0]['lmList'][8]
                else:
                    delx, dely, delz = startPos1[0] - hands[0]['lmList'][8][0], startPos1[1] - hands[0]['lmList'][8][1], startPos1[2] - hands[0]['lmList'][8][2]
        # 旋转
        if detector.fingersUp(hands[0]) == [1, 1, 1, 1, 0]:
            dis12, _, _ = detector.findDistance(hands[0]['lmList'][8][0:2], hands[0]['lmList'][12][0:2], img)
            dis23, _, _ = detector.findDistance(hands[0]['lmList'][12][0:2], hands[0]['lmList'][16][0:2], img)
            if dis12 < 50 and dis23 < 50:
                flag = 2
                if startPos2 is None:
                    startPos2 = hands[0]['lmList'][8]
                else:
                    delrx, delry = startPos2[0] - hands[0]['lmList'][8][0], startPos2[1] - hands[0]['lmList'][8][1]
        if detector.fingersUp(hands[0]) == [1, 1, 1, 1, 1]:
            dis12, _, _ = detector.findDistance(hands[0]['lmList'][8][0:2], hands[0]['lmList'][12][0:2], img)
            dis23, _, _ = detector.findDistance(hands[0]['lmList'][12][0:2], hands[0]['lmList'][16][0:2], img)
            if dis12 < 50 and dis23 < 50:
                flag = 3
                if startPos3 is None:
                    startPos3 = hands[0]['lmList'][8]
                else:
                    delrz = startPos3[0] - hands[0]['lmList'][8][0]
    
    if len(hands) == 2:
        # 放缩
        if detector.fingersUp(hands[0]) == [0, 1, 0, 0, 0] and detector.fingersUp(hands[1]) == [0, 1, 0, 0, 0]:
            flag = 4
            lmList1 = hands[0]["lmList"]
            lmList2 = hands[1]["lmList"]
            length, info, img = detector.findDistance(lmList1[8][0:2], lmList2[8][0:2], img)
            if startDistance is None:
                startDistance = length
            else:
                scale = (int)(length - startDistance)//45

    if flag != 1:
        startPos1 = None
        cx, cy, cz = newx, newy, newz
        delx, dely, delz = 0, 0, 0
    if flag != 2:
        startPos2 =None
        rx, ry = newrx, newry
        delrx, delry = 0, 0
    if flag != 3:
        startPos3 =None
        rz = newrz
        delrz = 0
    if flag != 4:
        startDistance = None
        W, H, L = newW, newH, newL
        scale = 0

    newx ,newy, newz = cx + delx, cy + dely, cz + delz
    newrx ,newry, newrz = rx + delrx, ry + delry, rz + delrz
    newW, newH, newL = W + scale, H + scale, L + scale
    
    data = []
    data.extend([flag])
    data.extend([newx, height - newy, newz])
    data.extend([newrx, newry, newrz])
    data.extend([newW, newH, newL])
    sock.sendto(str.encode(str(data)), serverAddressPort)
    
    cv2.imshow("Image", img)
    cv2.waitKey(1)